In [35]:
import json
import os
from openai import AzureOpenAI

config_path = 'config.json'
with open(config_path, 'r') as config_file:
    config = json.load(config_file)

azure_api_key = config['azure_api_key']
azure_api_version = config['azure_api_version']
azure_endpoint = config['azure_endpoint']
deployment_name = config['deployment_name']

client = AzureOpenAI(
    api_key=azure_api_key,  
    api_version=azure_api_version,
    azure_endpoint=azure_endpoint
)

def generate_text(prompt):

    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,
        temperature=0.7
    )

    text = response.choices[0].message.content.strip()

    return text

def generate_response(feedback):
    prompt = f"Generate a short but sympathetic response to the user about the feedback {feedback}. Is the review negative or positive? Write a N for negative or P for positive at the end. Mixed reviews are considered negative. Don't write anything else or break format"
    response = generate_text(prompt)
    sentiment = response[-1]
    response = response[:-1].strip()
    category = "Positive"
    severity = "0"
    if sentiment == 'N':
        prompt_2 = f"For the given negative feedback {feedback}, specify which category (service, food, etc.) was negative under heading 'Category:' and the severity (1 to 5 range) of the feedback under heading 'Severity:'. Don't write anything else or break format."
        text = generate_text(prompt_2)
        parts = text.split("\n")
        for part in parts:
            if part.startswith("Category:"):
                category = part.replace("Category:", "").strip()
            elif part.startswith("Severity:"):
                severity = part.replace("Severity:", "").strip()
    return response, category, severity




In [36]:
# Example
feedback = "The service was good but the food not so much. It was really cold. But the waiter was so polite."
response, category, severity = generate_response(feedback)
print("Response to feedback:")
print(response)
print("Category:", category)
print("Severity:", severity)



Response to feedback:
I'm sorry to hear that the food didn't meet your expectations, but I'm glad you had a positive experience with our service. We'll work on improving the food quality.
Category: food
Severity: 3
